In [1]:

import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tqdm import tqdm_notebook 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import itertools
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, concatenate, Input, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, Adadelta
from keras.layers.advanced_activations import LeakyReLU
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
SPECIES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen',
              'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse',
              'Small-flowered Cranesbill', 'Sugar beet']

In [3]:
data_dir = r"Segmented"

In [4]:
for species in SPECIES:
    print('{} {} images'.format(species, len(os.listdir(os.path.join(data_dir, species)))))

Black-grass 333 images
Charlock 454 images
Cleavers 348 images
Common Chickweed 716 images
Common wheat 258 images
Fat Hen 543 images
Loose Silky-bent 819 images
Maize 260 images
Scentless Mayweed 608 images
Shepherds Purse 276 images
Small-flowered Cranesbill 580 images
Sugar beet 464 images


In [5]:
train = []

for species_num, species in enumerate(SPECIES):
    for file in os.listdir(os.path.join(data_dir, species)):
        train.append(['Segmented/{}/{}'.format(species, file), species_num, species])

In [6]:
train = pd.DataFrame(train, columns=['file', 'species_num', 'species'])

print('Training Data: ',train.shape)

Training Data:  (5659, 3)


In [7]:
Input_images = []
for file in tqdm_notebook(train['file'],total=len(train)):
    path = cv2.imread(file)
    Input_images.append(cv2.resize(path,dsize=(150,150)))
#plt.imshow(Input_image)
#plt.title('Input image, Shape: '+str(Input_image.shape))
#plt.show()
#X_train = X_train.reshape(-1,28,28,1)
#x_test = X_test.reshape(-1,28,28,1)

In [8]:
Input_images = np.array(Input_images)

In [9]:
Input_images.shape

(5659, 150, 150, 3)

In [10]:
labels = train['species_num']
labels = to_categorical(labels, num_classes = len(SPECIES))

In [11]:
type(Input_images)

numpy.ndarray

In [12]:
X_train, X_test, y_train, y_test = train_test_split(Input_images,labels,test_size = 0.1, random_state=10)

In [13]:
type(X_train),type(X_test),type(y_test),type(y_train)

(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)

In [14]:
X_train.shape,y_train.shape

((5093, 150, 150, 3), (5093, 12))

In [40]:
tf.reset_default_graph()

In [41]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(150,150,3),
                                               include_top=False,
                                               weights='imagenet')

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [42]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 151, 151, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 75, 75, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 75, 75, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

block_4_expand_relu (ReLU)      (None, 19, 19, 192)  0           block_4_expand_BN[0][0]          
__________________________________________________________________________________________________
block_4_depthwise (DepthwiseCon (None, 19, 19, 192)  1728        block_4_expand_relu[0][0]        
__________________________________________________________________________________________________
block_4_depthwise_BN (BatchNorm (None, 19, 19, 192)  768         block_4_depthwise[0][0]          
__________________________________________________________________________________________________
block_4_depthwise_relu (ReLU)   (None, 19, 19, 192)  0           block_4_depthwise_BN[0][0]       
__________________________________________________________________________________________________
block_4_project (Conv2D)        (None, 19, 19, 32)   6144        block_4_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_4_pr

__________________________________________________________________________________________________
block_8_project (Conv2D)        (None, 10, 10, 64)   24576       block_8_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_8_project_BN (BatchNormal (None, 10, 10, 64)   256         block_8_project[0][0]            
__________________________________________________________________________________________________
block_8_add (Add)               (None, 10, 10, 64)   0           block_7_add[0][0]                
                                                                 block_8_project_BN[0][0]         
__________________________________________________________________________________________________
block_9_expand (Conv2D)         (None, 10, 10, 384)  24576       block_8_add[0][0]                
__________________________________________________________________________________________________
block_9_ex

__________________________________________________________________________________________________
block_13_expand_BN (BatchNormal (None, 10, 10, 576)  2304        block_13_expand[0][0]            
__________________________________________________________________________________________________
block_13_expand_relu (ReLU)     (None, 10, 10, 576)  0           block_13_expand_BN[0][0]         
__________________________________________________________________________________________________
block_13_pad (ZeroPadding2D)    (None, 11, 11, 576)  0           block_13_expand_relu[0][0]       
__________________________________________________________________________________________________
block_13_depthwise (DepthwiseCo (None, 5, 5, 576)    5184        block_13_pad[0][0]               
__________________________________________________________________________________________________
block_13_depthwise_BN (BatchNor (None, 5, 5, 576)    2304        block_13_depthwise[0][0]         
__________

In [43]:
base_model.trainable = False

In [44]:
model = tf.keras.Sequential([
  base_model])

In [45]:
#trainable model place

In [46]:
model.add(Flatten())

In [47]:
model.add(Dense(500,'relu'))

In [48]:
model.add(Dense(200,'relu'))

In [49]:
model.add(Dense(12,'softmax'))

In [50]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [51]:
model.fit(
    X_train,
    y_train,
    batch_size=256,
    epochs=2,
    validation_split=0.2
)

Train on 4583 samples, validate on 510 samples
Epoch 1/2
4583/4583 [==============================] - ETA: 3:56 - loss: 3.3104 - acc: 0.070 - ETA: 3:35 - loss: 3.0408 - acc: 0.095 - ETA: 3:20 - loss: 2.8776 - acc: 0.109 - ETA: 3:08 - loss: 2.7707 - acc: 0.127 - ETA: 2:53 - loss: 2.6929 - acc: 0.149 - ETA: 2:39 - loss: 2.6080 - acc: 0.173 - ETA: 2:25 - loss: 2.5271 - acc: 0.197 - ETA: 2:12 - loss: 2.4676 - acc: 0.214 - ETA: 1:58 - loss: 2.4069 - acc: 0.237 - ETA: 1:44 - loss: 2.3642 - acc: 0.252 - ETA: 1:31 - loss: 2.3155 - acc: 0.267 - ETA: 1:19 - loss: 2.2714 - acc: 0.281 - ETA: 1:06 - loss: 2.2254 - acc: 0.297 - ETA: 53s - loss: 2.1790 - acc: 0.316 - ETA: 40s - loss: 2.1455 - acc: 0.32 - ETA: 26s - loss: 2.1128 - acc: 0.33 - ETA: 12s - loss: 2.0776 - acc: 0.34 - 256s 56ms/sample - loss: 2.0448 - acc: 0.3581 - val_loss: 2.3933 - val_acc: 0.2490
Epoch 2/2
4583/4583 [==============================] - ETA: 4:05 - loss: 1.2811 - acc: 0.613 - ETA: 3:45 - loss: 1.2403 - acc: 0.652 - ETA: 3: